In [23]:
from aes import AES
from PIL import Image
import os
import random
from math import log, floor
import uuid

In [24]:
key = os.urandom(16)
print(f"key: {key}")
iv = os.urandom(16)
print(f"iv: {iv}")

key: b':\xbb\xbc\xcd\x00\x9b\xa6\r\xabn\xe3H\xa0MV\xf4'
iv: b',\x1e\x7f\xe9Fj3\x91\t\x8e\xb5/\xb7.\xb1A'


In [27]:
def create_rgb_arrays(height, width, pixel_data):
    r_arr = []
    g_arr = []
    b_arr = []
    for y in range(height):
        for x in range(width):
            r,g,b = pixel_data[x,y]
            r_arr.append(r)
            g_arr.append(g)
            b_arr.append(b)
    return r_arr, g_arr, b_arr

def encrypt_array(arr, key, iv,mode: int):
    bytes_arr = bytearray(arr)
    encrypted_bytes:bytes = b""
    aes = AES(key)
    encrypted_method = {
        0: aes.encrypt,
        1: aes.encrypt_cbc,
        2: aes.encrypt_pcbc,
        3: aes.encrypt_cfb,
        4: aes.encrypt_ofb,
    }
    if mode not in encrypted_method.keys():
        raise ValueError(f"Invalid mode: {mode}")
    if mode == 0:
        encrypted_bytes = encrypted_method[mode](bytes_arr)
    else:
        encrypted_bytes = encrypted_method[mode](bytes_arr,iv)
    return encrypted_bytes

def create_rgb_encrypted_img(path:str,key:bytes,mode:int,iv:bytes=b''):
    with Image.open(path) as im:
        pixel_data = im.load()

    width, height = im.size
    print(f"width: {width}, height: {height}")    
    if(width < 16):
        raise ValueError("Can not encrypt the image with width less than 16")
    r_arr, g_arr, b_arr = create_rgb_arrays(height, width, pixel_data)
    
    r_encrypted = encrypt_array(r_arr, key, iv,mode)
    g_encrypted = encrypt_array(g_arr, key, iv,mode)
    b_encrypted = encrypt_array(b_arr, key, iv,mode)

    print("encrypted bytes len: ", r_encrypted.__len__())
    print("plain bytes len: ",r_arr.__len__())
    print("needed width: ", width - (r_encrypted.__len__() - r_arr.__len__()))
    
    need_width = width - (r_encrypted.__len__() - r_arr.__len__())
    
    need_width_exp = floor(log(need_width,255))
    need_width_quo = need_width // pow(255,need_width_exp)
    need_width_rem = need_width % pow(255,need_width_exp)

    if(need_width_exp >255 | need_width_quo > 255 | need_width_rem > 255):
        raise ValueError("Can not encrypt the image")
    print("add_pixel: ", (need_width_exp,need_width_quo,need_width_rem))

    encrypted_img = Image.new("RGB", (width, height+1))
    for y in range(height+1):
        for x in range(width):
            if x+y*width < r_encrypted.__len__():
                encrypted_img.putpixel((x,y), (r_encrypted[x+y*width],g_encrypted[x+y*width],b_encrypted[x+y*width]))
            elif  x == width-1 and y == height:
                encrypted_img.putpixel((x,y), (need_width_exp,need_width_quo,need_width_rem))
            else:
                encrypted_img.putpixel((x,y), (random.randint(0,255),random.randint(0,255),random.randint(0,255)))
    return encrypted_img


In [29]:
path = str(input("Enter image path: "))
print("Select mode of operation:")
print("0. No mode of operation")
print("1. CBC")
print("2. PCBC")
print("3. CFB")
print("4. OFB")
mode = int(input("Enter mode: "))
encrypt_path = "img/" + str(uuid.uuid4()) + ".png"
print(f"encrypt_path: {encrypt_path}")
if(mode != 0):
    encrypted_img = create_rgb_encrypted_img(path,key,mode,iv)
else:
    encrypted_img = create_rgb_encrypted_img(path,key,mode)
encrypted_img.save(encrypt_path)
print("encrypted image saved")


Select mode of operation:
0. No mode of operation
1. CBC
2. PCBC
3. CFB
4. OFB
encrypt_path: img/1465f610-a683-4660-a0c8-2996aee29bf6.png
width: 1200, height: 1770
encrypted bytes len:  2124016
plain bytes len:  2124000
needed width:  1184
add_pixel:  (1, 4, 164)
encrypted image saved


In [6]:

def decrypt_array(arr, key, iv, mode: int):
    bytes_arr = bytearray(arr)
    aes = AES(key)

    decrypt_methods = {
        0: aes.decrypt,
        1: aes.decrypt_cbc,
        2: aes.decrypt_pcbc,
        3: aes.decrypt_cfb,
        4: aes.decrypt_ofb,
    }

    if mode not in decrypt_methods.keys():
        raise ValueError(f"Invalid mode: {mode}")

    decrypted_bytes = decrypt_methods[mode](bytes_arr, iv)
    return decrypted_bytes
def decrypt_encrypted_img(path:str, key:bytes, iv:bytes,mode:int):
    with Image.open(path) as encrypted_img:
        width,height = encrypted_img.size
        pixel_data = encrypted_img.load()
    print(f"width: {width} height: {height}")
    need_width_exp,need_width_quo,need_width_rem = pixel_data[width-1,height-1]
    print(f"need_width_exp: {need_width_exp}, need_width_quo: {need_width_quo}, need_width_rem: {need_width_rem}")
    need_width = need_width_rem + need_width_quo*pow(255,need_width_exp)
    print("need_width: ", need_width)

    r_encrypted_bytes = []
    g_encrypted_bytes = []
    b_encrypted_bytes = []

    for y in range(height):
        for x in range(width):
            r,g,b = pixel_data[x,y]
            if x+y*width < width*height - need_width:
                r_encrypted_bytes.append(r)
                g_encrypted_bytes.append(g)
                b_encrypted_bytes.append(b)
            else:
                break
    
    r_decrypted_bytes = decrypt_array(r_encrypted_bytes, key, iv,mode)
    g_decrypted_bytes = decrypt_array(g_encrypted_bytes, key, iv,mode)
    b_decrypted_bytes = decrypt_array(b_encrypted_bytes, key, iv,mode)

    decrypt_width = width
    decrypt_height = height - 1 if need_width > 0 else height
    decrypted_img = Image.new("RGB", (decrypt_width, decrypt_height))

    for y in range(decrypt_height):
        for x in range(decrypt_width):
            decrypted_img.putpixel((x,y),(r_decrypted_bytes[x+y*width],g_decrypted_bytes[x+y*width],b_decrypted_bytes[x+y*width]))
            
    return decrypted_img


In [11]:

encrypt_path = str(input("Enter encrypted image path: "))
encrypt_path_name = ''.join(encrypt_path.split('.png'))

decrypt_path = f"{encrypt_path_name}_decrypted.png"
print(f"decrypt_path: {decrypt_path}")

print("Select mode of operation:")
print("0. No mode of operation")
print("1. CBC")
print("2. PCBC")
print("3. CFB")
print("4. OFB")
mode = int(input("Enter mode: "))

decrypted_img = decrypt_encrypted_img(encrypt_path,key,iv,mode)
decrypted_img.save(decrypt_path)
print("decrypted image saved")


decrypt_path: img/d9c07831-c27b-42ee-b611-d917ce85219d_decrypted.png
Select mode of operation:
0. No mode of operation
1. CBC
2. PCBC
3. CFB
4. OFB
width: 800 height: 451
need_width_exp: 1, need_width_quo: 3, need_width_rem: 35
need_width:  800
decrypted image saved
